In [50]:
!pip install -q tensorflow-data-validation

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tensorflow_data_validation.statistics import stats_options as options

In [2]:
pd.options.display.float_format = '{:,.3f}'.format

In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/diabetes_classification/diabetes_data.csv'
data = pd.read_csv(path, delimiter=';')

In [4]:
data.head()

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,class
0,40,Male,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,Male,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,Male,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,Male,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,Male,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1


In [5]:
data['gender'] = data['gender'].apply(lambda x: 1 if x=='Female' else 0)

In [6]:
data['class'].value_counts(normalize=True)

1   0.615
0   0.385
Name: class, dtype: float64

In [7]:
def do_chi2(df, cols, target):
  alpha = 0.05
  # tabular value for DOF = 1 and  alpha=0.05
  critical_value = 3.841

  significant_cols = []

  for col in cols:
    contingency_table = pd.crosstab(df[col], df[target])
    stat, p_val, __, __ = chi2_contingency(contingency_table)
    
    print(f"For {col} p_val: {p_val}, chi2_stat: {round(stat,2)}, critical value: {critical_value}")
    
    if p_val < alpha or stat > critical_value:
      print("Reject the H0. Variable is significant.")
      significant_cols.append(col)
    else:
      print('Fail to reject the H0.')
    
  return significant_cols

In [8]:
columns_for_chi2 = list(data.drop(['age','class'], axis=1).columns)

In [9]:
significant_cols = do_chi2(data, columns_for_chi2, 'class')

For gender p_val: 3.289703730553294e-24, chi2_stat: 103.04, critical value: 3.841
Reject the H0. Variable is significant.
For polyuria p_val: 1.7409117803442155e-51, chi2_stat: 227.87, critical value: 3.841
Reject the H0. Variable is significant.
For polydipsia p_val: 6.1870096408863144e-49, chi2_stat: 216.17, critical value: 3.841
Reject the H0. Variable is significant.
For sudden_weight_loss p_val: 5.969166262549937e-23, chi2_stat: 97.3, critical value: 3.841
Reject the H0. Variable is significant.
For weakness p_val: 4.869843446585542e-08, chi2_stat: 29.77, critical value: 3.841
Reject the H0. Variable is significant.
For polyphagia p_val: 1.1651584346409135e-14, chi2_stat: 59.6, critical value: 3.841
Reject the H0. Variable is significant.
For genital_thrush p_val: 0.016097902991938178, chi2_stat: 5.79, critical value: 3.841
Reject the H0. Variable is significant.
For visual_blurring p_val: 1.7015036753241196e-08, chi2_stat: 31.81, critical value: 3.841
Reject the H0. Variable is s

In [11]:
scaler = MinMaxScaler()
data['age'] = scaler.fit_transform(np.array(data['age']).reshape((-1,1)))

In [12]:
dataset = data[significant_cols + ['age','class']]

In [13]:
dataset.head()

,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,irritability,partial_paresis,muscle_stiffness,alopecia,age,class
0,0,0,1,0,1,0,0,0,0,0,1,1,0.324,1
1,0,0,0,0,1,0,0,1,0,1,0,1,0.568,1
2,0,1,0,0,1,1,0,0,0,0,1,1,0.338,1
3,0,0,0,1,1,1,1,0,0,0,0,0,0.392,1
4,0,1,1,1,1,1,0,1,1,1,1,1,0.595,1


In [14]:
X = dataset[dataset.columns[:-1]].values
y = dataset[dataset.columns[-1]].values


In [15]:
X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state=0)

In [16]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=16, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
20/20 [==============================] - 1s 14ms/step - loss: 0.7023 - accuracy: 0.4872 - val_loss: 0.6546 - val_accuracy: 0.6442
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.6088 - accuracy: 0.6474 - val_loss: 0.5841 - val_accuracy: 0.6154
Epoch 3/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5411 - accuracy: 0.6571 - val_loss: 0.5299 - val_accuracy: 0.6538
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4927 - accuracy: 0.7468 - val_loss: 0.4825 - val_accuracy: 0.7692
Epoch 5/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4480 - accuracy: 0.8109 - val_loss: 0.4392 - val_accuracy: 0.8269
Epoch 6/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4048 - accuracy: 0.8942 - val_loss: 0.4007 - val_accuracy: 0.8846
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3661 - accuracy: 0.8974 - val_loss: 0.3688 - val_accuracy: 0.9135
Epoch 8/30
20/20 [=

In [19]:
result = model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.1907 - accuracy: 0.9423


In [20]:
result

[0.19068874418735504, 0.942307710647583]

# Tensorflow Data Validation

In [30]:
X_df = dataset[dataset.columns[:-1]]
y_df = dataset[dataset.columns[-1]]

X_df[significant_cols] = X_df[significant_cols].astype('bool')

X_train_df, X_temp_df, y_train_df, y_temp_df = train_test_split(X_df,y_df, test_size=0.4, random_state=0)
X_valid_df, X_test_df, y_valid_df, y_test_df = train_test_split(X_temp_df,y_temp_df, test_size=0.5, random_state=0)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [31]:
X_train_df_stats=tfdv.generate_statistics_from_dataframe(X_train_df)
schema = tfdv.infer_schema(statistics=X_train_df_stats)

In [42]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'gender',INT,required,,-
'polyuria',INT,required,,-
'polydipsia',INT,required,,-
'sudden_weight_loss',INT,required,,-
'weakness',INT,required,,-
'polyphagia',INT,required,,-
'genital_thrush',INT,required,,-
'visual_blurring',INT,required,,-
'irritability',INT,required,,-


In [43]:
X_test_df_stats=tfdv.generate_statistics_from_dataframe(X_test_df)

In [44]:
tfdv.visualize_statistics(lhs_statistics=X_test_df_stats, rhs_statistics=X_train_df_stats,lhs_name='TEST_DATASET', rhs_name='TRAIN_DATASET')

In [45]:
eval_anomalies = tfdv.validate_statistics(statistics=X_test_df_stats, schema=schema)

In [46]:
tfdv.display_anomalies(eval_anomalies)